In [1]:
!pip3 install --upgrade tensorflow 
!pip install keras
!pip install gspread oauth2client
!pip install unidecode
!pip install xlsxwriter

Requirement already up-to-date: tensorflow in /usr/local/lib/python3.7/dist-packages (2.4.1)
     |████████████████████████████████| 245kB 7.0MB/s 
     |████████████████████████████████| 153kB 8.2MB/s 


# Import các thư viện cần thiết

In [2]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers
import cv2
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

import keras 
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout, Flatten, AveragePooling2D, MaxPooling2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression

import joblib
from datetime import datetime
import time
import pytz

# trích xuất menu để phục vụ đọc ảnh

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
menu = []
os.getcwd()
collection = '/content/drive/MyDrive/Đồ án số 2 khoa học dữ liệu/Dataset/Test'
for i, mon in enumerate(os.listdir(collection)):
    menu.append(mon)

print('Số lượng: ', len(menu), ' món')
menu

Số lượng:  15  món


['BanhBao',
 'BanhBeo',
 'BanhCanhChaCa',
 'BanhMi',
 'BanhTrungThu',
 'BanhXeo',
 'BapXao',
 'BunBo',
 'BunDau',
 'ChaGio',
 'ComTam',
 'Mien',
 'MiQuang',
 'Pho',
 'XoiMan']

# Đọc ảnh

In [5]:
def get_data(path_folder, img_rows, img_cols):
    x = []
    y = []
    for mon in menu:
        folder_mon = path_folder + '/' + mon
        for num, filename in enumerate(os.listdir(folder_mon)):
            print(filename)
            img = cv2.imread(os.path.join(folder_mon, filename))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (img_rows, img_cols)) 
            if img is not None:
                x.append(img)
                y.append(mon)
    return x, y



In [6]:
x_train, y_train = get_data('/content/drive/MyDrive/Đồ án số 2 khoa học dữ liệu/Dataset/Train', 224,  224)
x_test, y_test = get_data('/content/drive/MyDrive/Đồ án số 2 khoa học dữ liệu/Dataset/Test', 224,  224)
x_cha, y_cha = get_data('/content/drive/MyDrive/Đồ án số 2 khoa học dữ liệu/Dataset/Challenge', 224,  224)

Streaming output truncated to the last 5000 lines.
Pho_000563.jpg
Pho_000564.jpg
Pho_000565.jpg
Pho_000566.jpg
Pho_000567.jpg
Pho_000568.jpg
Pho_000569.jpg
Pho_000570.jpg
Pho_000571.jpg
Pho_000572.jpg
Pho_000573.jpg
Pho_000574.jpg
Pho_000575.jpg
Pho_000576.jpg
Pho_000577.jpg
Pho_000578.jpg
Pho_000579.jpg
Pho_000580.jpg
Pho_000581.jpg
Pho_000582.jpg
Pho_000583.jpg
Pho_000584.jpg
Pho_000585.jpg
Pho_000586.jpg
Pho_000587.jpg
Pho_000588.jpg
Pho_000589.jpg
Pho_000590.jpg
Pho_000591.jpg
Pho_000592.jpg
Pho_000593.jpg
Pho_000594.jpg
Pho_000595.jpg
Pho_000596.jpg
Pho_000597.jpg
Pho_000598.jpg
Pho_000599.jpg
Pho_000600.jpg
Pho_000601.jpg
Pho_000602.jpg
Pho_000603.jpg
Pho_000604.jpg
Pho_000605.jpg
Pho_000606.jpg
Pho_000607.jpg
Pho_000608.jpg
Pho_000609.jpg
Pho_000610.jpg
Pho_000611.jpg
Pho_000612.jpg
Pho_000613.jpg
Pho_000614.jpg
Pho_000615.jpg
Pho_000616.jpg
Pho_000617.jpg
Pho_000618.jpg
Pho_000619.jpg
Pho_000620.jpg
Pho_000621.jpg
Pho_000622.jpg
Pho_000623.jpg
Pho_000624.jpg
Pho_000625.jpg
Pho_

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import tensorflow as tf
from keras.preprocessing import image_dataset_from_directory
import matplotlib.pyplot as plt

ds_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
lb.fit(y_train)
y_train_lb = lb.transform(y_train)
y_test_lb = lb.transform(y_test)
y_cha_lb = lb.transform(y_cha)


In [ ]:
np.save('classes.npy', lb.classes_)

# Chuẩn hóa ảnh

In [ ]:
x_train = np.array(x_train)
x_test = np.array(x_test)
x_cha = np.array(x_cha)

x_train = x_train.reshape(-1, 224, 224, 3)
x_test = x_test.reshape(-1, 224,224, 3)
x_cha = x_cha.reshape(-1, 224,224, 3)
Input_shape = (224, 224, 3)



x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_cha = x_cha.astype('float32')
x_train /= 255
x_test /= 255
x_cha /= 255

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Huấn luyện mô hình MobileNetV2

In [ ]:
from tensorflow.keras.applications import DenseNet121, MobileNetV2
model2 = MobileNetV2()
model2.summary()

In [ ]:
base_inputs = model2.layers[0].input
base_outputs = model2.layers[-2].output 
final_outputs =  layers.Dense(15, activation='softmax')(base_outputs)


new_model2 = keras.Model(inputs = base_inputs, outputs = final_outputs )
new_model2.summary()


In [ ]:
m = tf.keras.metrics.TopKCategoricalAccuracy(k=5)
opt = keras.optimizers.Adam(learning_rate=0.001)
new_model2.compile(optimizer=opt, loss = 'categorical_crossentropy', metrics = ['accuracy', m] )


In [ ]:
Batch_size = 64
Epochs = 15
Number_classes = 15

# Kết quả đánh giá
f1_macro_result = []
acc_result = []
confusion_result = []
count = 1
temp = 0
from sklearn.model_selection import StratifiedShuffleSplit
# K-Fold
kf = StratifiedShuffleSplit(n_splits = 5, test_size = 0.2)

for train_index, test_index in kf.split(x_train, y_train_lb):

  X_train, X_val = x_train[train_index], x_train[test_index]
  Y_train, Y_val = y_train_lb[train_index], y_train_lb[test_index]
  #X_test_raw = X_test

  

  # Run model
  new_model2.fit(X_train, keras.utils.to_categorical(Y_train, Number_classes), batch_size = Batch_size, epochs= Epochs, verbose = 1,
          validation_data = (X_val, keras.utils.to_categorical( Y_val, Number_classes)))
  # Predict 
  Y_val_pred = new_model2.predict(X_val)
  Y_val_pred =  Y_val_pred.argmax(axis = -1)

  # Confusion matrix
  confusion = confusion_matrix(Y_val,  Y_val_pred)
  confusion_result.append(confusion)

  # F1 macro score
  f1_macro = f1_score( Y_val,  Y_val_pred, average = 'macro')
  f1_macro_result.append(f1_macro)

  # Accuracy score
  acc = accuracy_score( Y_val,  Y_val_pred)
  acc = acc_result.append(acc)
  
  # Print Fold: Confusion matrix
  print('Fold {}: {}'.format(count, f1_macro))
  # print(confusion)
  # Print result
  print('Average confusion matrix: ')
  cm = np.mean(confusion_result, axis = -3)
  sb.heatmap(cm, annot = True, fmt = 'g', cmap = 'Blues')
  plt.xlabel('Predicted label')
  plt.ylabel('True label')
  plt.title('Confusion matrix')
  plt.show()
  print('Average F1 score macro: {}'.format(str(np.mean(f1_macro_result))))
  print('Average accuracy: {}'.format(str(np.mean(acc_result))))
  count +=1
  

In [ ]:
new_model2.save(f'/content/drive/MyDrive/Đồ án số 1 khoa học dữ liệu/Linh gán nhãn/Mobile net1.h5')
print('Model saved')

Model saved


# Huấn luyện mô hình Densenet121

In [ ]:
model1 = DenseNet121()
model1.summary()

In [ ]:
base_inputs = model1.layers[0].input
base_outputs = model1.layers[-2].output 
final_outputs =  layers.Dense(15, activation='softmax')(base_outputs)


new_model1 = keras.Model(inputs = base_inputs, outputs = final_outputs )
new_model1.summary()


In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.001)
new_model1.compile(optimizer=opt, loss = 'categorical_crossentropy', metrics = ['accuracy', m] )


In [ ]:
Batch_size = 64
Epochs = 15
Number_classes = 15

# Kết quả đánh giá
f1_macro_result = []
acc_result = []
confusion_result = []
count = 1
temp = 0
from sklearn.model_selection import StratifiedShuffleSplit
# K-Fold
kf = StratifiedShuffleSplit(n_splits = 5, test_size = 0.2)

for train_index, test_index in kf.split(x_train, y_train_lb):

  X_train, X_val = x_train[train_index], x_train[test_index]
  Y_train, Y_val = y_train_lb[train_index], y_train_lb[test_index]
  #X_test_raw = X_test

  

  # Run model
  new_model1.fit(X_train, keras.utils.to_categorical(Y_train, Number_classes), batch_size = Batch_size, epochs= Epochs, verbose = 1,
          validation_data = (X_val, keras.utils.to_categorical( Y_val, Number_classes)))
  # Predict 
  Y_val_pred = new_model1.predict(X_val)
  Y_val_pred =  Y_val_pred.argmax(axis = -1)

  # Confusion matrix
  confusion = confusion_matrix(Y_val,  Y_val_pred)
  confusion_result.append(confusion)

  # F1 macro score
  f1_macro = f1_score( Y_val,  Y_val_pred, average = 'macro')
  f1_macro_result.append(f1_macro)

  # Accuracy score
  acc = accuracy_score( Y_val,  Y_val_pred)
  acc = acc_result.append(acc)
  
  # Print Fold: Confusion matrix
  print('Fold {}: {}'.format(count, f1_macro))
  # print(confusion)
  # Print result
  print('Average confusion matrix: ')
  cm = np.mean(confusion_result, axis = -3)
  sb.heatmap(cm, annot = True, fmt = 'g', cmap = 'Blues')
  plt.xlabel('Predicted label')
  plt.ylabel('True label')
  plt.title('Confusion matrix')
  plt.show()
  print('Average F1 score macro: {}'.format(str(np.mean(f1_macro_result))))
  print('Average accuracy: {}'.format(str(np.mean(acc_result))))
  count +=1
  

# Load mô hình và kiểm tra kết quả trên MobileNetV2

In [ ]:
new_model2 = keras.models.load_model("/content/drive/MyDrive/Đồ án số 1 khoa học dữ liệu/Linh gán nhãn/Mobile net.h5")

## Kiểm tra trên tập test

In [ ]:
y_test_pre = new_model2.predict(x_test)
y_test_pre =  y_test_pre.argmax(axis = -1)
f1_score(y_test_lb,  y_test_pre,  average = 'macro')

In [ ]:
cm = confusion_matrix( y_test_lb,  y_test_pre)
plt.figure(figsize=(7, 7))
sb.heatmap(cm, annot = True, fmt = 'g', cmap = 'Blues', xticklabels = menu, yticklabels = menu)
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion matrix')
plt.savefig('mobilenet')
plt.show()

In [ ]:
new_model2.evaluate(
    x=x_test,
    y=keras.utils.to_categorical( y_test_lb, Number_classes),
    batch_size=64,
    verbose=1)

## Kiểm tra trên tập challenge

In [ ]:
y_cha_pre = new_model2.predict(x_cha)
y_cha_pre =  y_cha_pre.argmax(axis = -1)
print( 'Hệ số f1 score macro la: ', f1_score(y_cha_lb,  y_cha_pre,  average = 'macro'))

In [ ]:
cm = confusion_matrix( y_cha_lb,  y_cha_pre)
plt.figure(figsize=(7, 7))
sb.heatmap(cm, annot = True, fmt = 'g', cmap = 'Blues', xticklabels = menu, yticklabels = menu)
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion matrix')
plt.savefig('mobilenet')
plt.show()

In [ ]:
Number_classes = 15
new_model2.evaluate(
    x=x_cha,
    y=keras.utils.to_categorical( y_cha_lb, Number_classes),
    batch_size=64,
    verbose=1)

# Load mô hình densenet để kiểm tra 

## Kiểm tra trên tập test

In [ ]:
densenet = keras.models.load_model("/content/drive/MyDrive/Đồ án số 1 khoa học dữ liệu/Linh gán nhãn/Densetnet.h5")

In [ ]:
densenet.evaluate(
    x=x_test,
    y=keras.utils.to_categorical( y_test_lb, 15),
    batch_size=64,
    verbose=1)

In [ ]:
y_test_pre = densenet.predict(x_test)
y_test_pre =  y_test_pre.argmax(axis = -1)
f1_score(y_test_lb,  y_test_pre,  average = 'macro')

In [ ]:
cm = confusion_matrix( y_test_lb,  y_test_pre)
plt.figure(figsize=(7, 7))
sb.heatmap(cm, annot = True, fmt = 'g', cmap = 'Blues', xticklabels = menu, yticklabels = menu)
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion matrix')
plt.savefig('densenet')
plt.show()

## Kiểm tra trên tập challenge

In [ ]:
densenet.evaluate(
    x=x_cha,
    y=keras.utils.to_categorical( y_cha_lb, 15),
    batch_size=64,
    verbose=1)

In [ ]:
y_cha_pre = densenet.predict(x_cha)
y_cha_pre =  y_cha_pre.argmax(axis = -1)
print('Hệ số F1-score macro là: ', f1_score(y_cha_lb,  y_cha_pre,  average = 'macro'))